In [2]:
import pandas as pd

# Load the point cloud data
point_cloud_df = pd.read_csv('ground_truth.csv')
point_cloud_df.keys()

Index(['Points_0', 'Points_1', 'Points_2', 'Temperature'], dtype='object')

In [3]:
point_cloud_df = point_cloud_df.rename(columns={'Points_0': 'X', 'Points_1': 'Y', 'Points_2': 'Z'})
point_cloud_df.keys()

Index(['X', 'Y', 'Z', 'Temperature'], dtype='object')

In [7]:
min_x, max_x = point_cloud_df['X'].min(), point_cloud_df['X'].max()
min_y, max_y = point_cloud_df['Y'].min(), point_cloud_df['Y'].max()
min_z, max_z = point_cloud_df['Z'].min(), point_cloud_df['Z'].max()
min_x, max_x, min_y, max_y, min_z, max_z

(-1.0842e-19, 0.03, 0.0, 0.05, 0.0, 0.025)

# Uniform Sampling within the Bounding Box

In [6]:

import numpy as np

# Number of points to sample
n_samples = 50  # Example, adjust based on your needs

# Uniformly sample points
sampled_points = np.random.uniform([min_x, min_y, min_z], [max_x, max_y, max_z], (n_samples, 3))


# Rejection Sampling

In [8]:
from scipy.spatial import KDTree

# Build a KDTree for efficient nearest neighbor queries
point_cloud_kdtree = KDTree(point_cloud_df[['X', 'Y', 'Z']])

# Define a maximum acceptable distance from any point in the point cloud
max_distance = 0.05  # Adjust based on the scale of your domain

# Find the distance to the nearest point in the point cloud for each sampled point
distances, _ = point_cloud_kdtree.query(sampled_points)

# Filter the sampled points by distance, keeping only those within max_distance of the point cloud
accepted_points = sampled_points[distances < max_distance]

In [9]:
type(accepted_points), accepted_points.shape

(numpy.ndarray, (50, 3))

# Save as csv

In [10]:
# Find the index of the nearest neighbor in the point cloud for each accepted point
_, nearest_indices = point_cloud_kdtree.query(accepted_points)

# Use these indices to look up the corresponding temperatures in the point cloud
accepted_temperatures = point_cloud_df.iloc[nearest_indices]['Temperature'].values


In [11]:
import pandas as pd

# Create a DataFrame for the accepted points and their temperatures
accepted_df = pd.DataFrame(accepted_points, columns=['X', 'Y', 'Z'])
accepted_df['Temperature'] = accepted_temperatures

# Save to CSV
accepted_df.to_csv('accepted_points_with_temperature.csv', index=False)
